In [1]:
import torch

from network import Network
from data import get_MNIST_data_loaders
from train import train_network
from evaluate import accuracy

In [ ]:
device = torch.device("gpu" if torch.cuda.is_available() else "cpu")
print(f"{device=}")

variant = "p"
in_dim = 1 if variant == "p" else 28
hidden_dim = 8
out_dim = 10
batch_size = 256

train_loader, test_loader = get_MNIST_data_loaders(batch_size, variant=variant)
model = Network(
    in_dim, 
    hidden_dim, 
    out_dim, 
    neuron_type="ekfr", 
    freeze_neurons=False, 
    freeze_g=True
)

train_network(
    model, 
    train_loader, 
    epochs=30, 
    lr=0.005, 
    variant=variant,
    C=3
)

train_acc = accuracy(model, train_loader, variant=variant)
test_acc = accuracy(model, test_loader, variant=variant)
print(f"Train accuracy: {train_acc} | Test accuracy: {test_acc}")

device=device(type='cpu')


100%|█████████████████████████████████████████| 235/235 [02:41<00:00,  1.46it/s]


Epoch 1 | Loss: 539.6539916992188


100%|█████████████████████████████████████████| 235/235 [02:05<00:00,  1.88it/s]


Epoch 2 | Loss: 514.1881713867188


...

Epoch 5 | Loss: 439.1002502441406


100%|█████████████████████████████████████████| 235/235 [02:08<00:00,  1.83it/s]


Epoch 6 | Loss: 430.1212158203125


100%|█████████████████████████████████████████| 235/235 [02:09<00:00,  1.82it/s]


Epoch 7 | Loss: 420.11090087890625


In [ ]:
{
    "model_state_dict": model.to(torch.device("cpu")).state_dict(),
    "train_accuracy": train_acc,
    "test_accuracy": test_acc
}

In [ ]:
model.hidden_neurons.a.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(list(range(model.hidden_neurons.k)), model.hidden_neurons.a.detach()[4,:]) 